# Times

## Introduction

The concept of **time** is fundamental to any kind of time series analysis and simulation. This is why we have dedicated a lot of thought to its representation in thalesians.tsa.

Before we present some of these ideas, we need to make sure that thalesians.tsa appears on the Python path and import the relevant modules:

In [1]:
import os, sys
sys.path.append(os.path.abspath('../../main/python'))

from thalesians.tsa.simulation import xtimes, times

## `xtimes`

`xtimes` is a generator (and, by implication, an iterator; you can learn more about generators and iterators [here](https://wiki.python.org/moin/Generators)) designed to be a source of times to thalesians.tsa routines.

In [2]:
for t in xtimes(0, 5): print(t)

0
1
2
3
4


Since `xtimes` is a generator, the times are computed lazily:

In [3]:
xtimes(0, 5)

<generator object xtimes at 0x00000210CB9091A8>

To get hold of them all at once, you need to use something like

In [4]:
list(xtimes(0, 5))

[0, 1, 2, 3, 4]

or the shortcut

In [5]:
times(0, 5)

[0, 1, 2, 3, 4]

which amounts to the same thing. So far, we haven't seen anything that would justify `xtimes`'s existence: after all, we could have achieved the same result with Python's native `range`:

In [6]:
list(range(0, 5))

[0, 1, 2, 3, 4]

In order to understand why we may need `xtimes` and not `range` we need to delve deeper into the semantics of `xtimes`.

`xtimes` takes three arguments: `start`, `stop`, and `step`. `stop` is, in fact, optional (may be `None`), so `xtimes` enables us to set up a (theoretically) infinite number of timesteps:

In [7]:
ts = []
for t in xtimes(start=1):
    ts.append(t)
    if len(ts) == 5: break
ts

[1, 2, 3, 4, 5]

Perhaps more importantly, `start` and `stop` don't have to be `int`s. They may be `float`s, `date`s, `time`s, `datetime`s. Respectively, `stop` may be an `int`, `float`, or a `timedelta`, for example:

In [8]:
times(start=-3., stop=5., step=2.5)

[-3.0, -0.5, 2.0, 4.5]

In [9]:
import datetime as dt
times(dt.date(2017, 5, 5), dt.date(2017, 5, 10))

[datetime.date(2017, 5, 5),
 datetime.date(2017, 5, 6),
 datetime.date(2017, 5, 7),
 datetime.date(2017, 5, 8),
 datetime.date(2017, 5, 9)]

(By default, the `step` is `1`, `1.`, or `timedelta(days=1)`, depending on the types of `start` and `stop`.)

In [10]:
times(dt.time(8), dt.time(12), dt.timedelta(minutes=30))

[datetime.time(8, 0),
 datetime.time(8, 30),
 datetime.time(9, 0),
 datetime.time(9, 30),
 datetime.time(10, 0),
 datetime.time(10, 30),
 datetime.time(11, 0),
 datetime.time(11, 30)]

In [11]:
times(dt.datetime(2017, 5, 10), dt.datetime(2017, 5, 5), dt.timedelta(days=-1))

[datetime.datetime(2017, 5, 10, 0, 0),
 datetime.datetime(2017, 5, 9, 0, 0),
 datetime.datetime(2017, 5, 8, 0, 0),
 datetime.datetime(2017, 5, 7, 0, 0),
 datetime.datetime(2017, 5, 6, 0, 0)]

The flexibility of `xtimes`/`times` enables one to represent time using a data type that is suitable for the particular simulation and/or modelling task.

## Random times
The power of `xtimes` extends beyond its flexibility with times. Often one needs to deal with times that aren't regularly spaced, such as those arising from a homogeneous Poisson point process. Since the `step` parameter of `xtimes` is allowed to be a callable, this can be implemented as follows:

In [12]:
import thalesians.tsa.random as rnd
times(0., 10., step=lambda x: rnd.exponential(2.5))

[0.0, 1.1731702249421478, 8.69847380223595]

(Recall that the lengths of times between the occurrences of a Poisson process are exponentially distributed.)

The type flexibility of the function wrappers found in `thalesians.tsa.random` enable one to specify the parameter of the `exponential` distribution as a `timedelta`:

In [13]:
times(dt.datetime(2017, 5, 5, 8),
      dt.datetime(2017, 5, 5, 12),
      lambda x: rnd.exponential(dt.timedelta(minutes=30)))

[datetime.datetime(2017, 5, 5, 8, 0),
 datetime.datetime(2017, 5, 5, 8, 5, 5, 324767),
 datetime.datetime(2017, 5, 5, 8, 10, 10, 598092),
 datetime.datetime(2017, 5, 5, 8, 11, 58, 307875),
 datetime.datetime(2017, 5, 5, 9, 12, 18, 523431),
 datetime.datetime(2017, 5, 5, 9, 39, 52, 871308),
 datetime.datetime(2017, 5, 5, 10, 16, 49, 121419),
 datetime.datetime(2017, 5, 5, 10, 17, 26, 560173)]